# Create patient-level snapshots

## About snapshots

`patientflow` is organised around the following concepts:

- Prediction time: A moment in the day at which predictions are to be made, for example 09:30.
- Patient snapshot: A summary of data from the EHR capturing is known about a single patient at the prediction time. Each patient snapshot has a date and a prediction time associated with it.
- Group snaphot: A set of patients snapshots. Each group snapshot has a date and a prediction time associated with it.
- Prediction window: A period of hours that begins at the prediction time.

Its intended use is with data on hospital visits that are unfinished, to predict whether some outcome (admission from A&E, discharge from hospital, or transfer to another clinical specialty) will happen within the prediction window. What the outcome is doesn't really matter; the same methods can be used. 

The utility of our approach - and the thing that makes it very generalisable - is that we then build up from the patient-level predictions into a predictions for a whole cohort of patients at a point in time. That step is what creates useful information for bed managers. I show this in later notebooks.

To use `patientflow` your data should be in snapshot form. Here I suggest how to you might prepare your data, starting from past hospital visits that have already finished. I'm going to use some fake data on Emergency Department visits, and imagine that we want to predict whether a patient will be admitted to a ward after they leave the ED, or discharged from hospital. 

NOTE: In practice, how to *whether a patient was admitted after the ED visit*, and *when they were ready to be admitted*, can be tricky. How do you account for the fact that the patient may wait in the ED for a bed, after they were ready to leave? Likewise, if you were trying to predict discharge at the end of a hospital visit, should that that be the time they were ready to leave, or the time they actually left? Discharge delays are common, due to waiting for medication or transport, or waiting for onward care provision to become available. 

The outcome that you are aiming for will depend on your setting. You may have to infer when a patient was ready from available data. Suffice to say, think carefully about what it is you are trying to predict, and how you will identify the outcome in data. 

## Creating fake finished visits

I'll start by loading some fake data resembling the structure of EHR data on Emergency Department (ED) visits. In my fake data, each visit has one row, with an arrival time at the ED, a discharge time from the ED, the patient's age and an outcome of whether they were admitted after the ED visit. 

The `is_admitted` column is our label, indicating the outcome in this imaginary case. 

In [121]:
# Reload functions every time
%load_ext autoreload 
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [146]:
from patientflow.generate import patient_visits
visits_df, _, _ = patient_visits('2023-01-01', '2023-04-01', 25)

print(f'There are {len(visits_df)} visits in the fake dataset, with arrivals between {visits_df.arrival_datetime.min().date()} and {visits_df.arrival_datetime.max().date()} inclusive.')
visits_df.head()

There are 2253 visits in the fake dataset, with arrivals between 2023-01-01 and 2023-03-31 inclusive.


,patient_id,visit_number,arrival_datetime,departure_datetime,is_admitted,age
0,1658,14,2023-01-01 03:31:47,2023-01-01 08:00:47,0,30
1,238,20,2023-01-01 04:25:57,2023-01-01 07:43:57,1,61
2,354,1,2023-01-01 05:21:43,2023-01-01 08:52:43,1,86
3,114,3,2023-01-01 08:01:26,2023-01-01 09:38:26,0,33
4,497,10,2023-01-01 08:20:52,2023-01-01 11:20:52,0,59


## Create snapshots from fake data

My goal is to create snapshots of these visits. First, I define the times of day I will be issuing predictions at. 

In [147]:
prediction_times = [(6, 0), (9, 30), (12, 0), (15, 30), (22, 0)] # each time is expressed as a tuple of (hour, minute)

Then using the code below I create an array of all the snapshot dates in some date range that my data covers.

In [148]:
from datetime import datetime, time, timedelta, date

# Create date range
snapshot_dates = []
start_date = date(2023, 1, 1)
end_date = date(2023, 4, 1)

current_date = start_date
while current_date < end_date:
    snapshot_dates.append(current_date)
    current_date += timedelta(days=1)

print('First ten snapshot dates')
snapshot_dates[0:10]

First ten snapshot dates


[datetime.date(2023, 1, 1),
 datetime.date(2023, 1, 2),
 datetime.date(2023, 1, 3),
 datetime.date(2023, 1, 4),
 datetime.date(2023, 1, 5),
 datetime.date(2023, 1, 6),
 datetime.date(2023, 1, 7),
 datetime.date(2023, 1, 8),
 datetime.date(2023, 1, 9),
 datetime.date(2023, 1, 10)]

Next I iterate through the date array, using the arrival and departure times from the hospital visits table to identify any patients who were in the ED at the prediction time (eg 09:30 or 12.00 on each date). 

In [149]:
import pandas as pd


# Create empty list to store results for each snapshot date
patient_shapshot_list = []

# For each combination of date and time
for date_val in snapshot_dates:
    for hour, minute in prediction_times:
        snapshot_datetime = datetime.combine(date_val, time(hour=hour, minute=minute))

        # Filter dataframe for this snapshot
        mask = (visits_df["arrival_datetime"] <= snapshot_datetime) & (
            visits_df["departure_datetime"] > snapshot_datetime
        )
        snapshot_df = visits_df[mask].copy()

        # Skip if no patients at this time
        if len(snapshot_df) == 0:
            continue

        # Add snapshot information columns
        snapshot_df["snapshot_date"] = date_val
        snapshot_df["prediction_time"] = [(hour, minute)] * len(snapshot_df)
        
        patient_shapshot_list.append(snapshot_df)

# Combine all results into single dataframe
snapshots_df = pd.concat(patient_shapshot_list, ignore_index=True)

# Name the index snapshot_id
snapshots_df.index.name = "snapshot_id"

Note that each record in the snapshots dataframe is indexed by a unique snapshot_id. 

In [150]:
snapshots_df.head()

,patient_id,visit_number,arrival_datetime,departure_datetime,is_admitted,age,snapshot_date,prediction_time
snapshot_id,,,,,,,,
0,1658,14,2023-01-01 03:31:47,2023-01-01 08:00:47,0,30,2023-01-01,"(6, 0)"
1,238,20,2023-01-01 04:25:57,2023-01-01 07:43:57,1,61,2023-01-01,"(6, 0)"
2,354,1,2023-01-01 05:21:43,2023-01-01 08:52:43,1,86,2023-01-01,"(6, 0)"
3,114,3,2023-01-01 08:01:26,2023-01-01 09:38:26,0,33,2023-01-01,"(9, 30)"
4,497,10,2023-01-01 08:20:52,2023-01-01 11:20:52,0,59,2023-01-01,"(9, 30)"


Some patients are present at more than one of the prediction times, given them more than one entry in snapshots_df

In [151]:
snapshots_df.visit_number.value_counts()

visit_number
1784    4
1342    3
1432    3
1292    3
1604    3
       ..
777     1
787     1
774     1
770     1
2238    1
Name: count, Length: 1675, dtype: int64

In [152]:
# Displaying the snapshots for a visit with multiple snapshots
example_visit_number = snapshots_df.visit_number.value_counts().index[0]
snapshots_df[snapshots_df.visit_number == example_visit_number]


,patient_id,visit_number,arrival_datetime,departure_datetime,is_admitted,age,snapshot_date,prediction_time
snapshot_id,,,,,,,,
1512,459,1784,2023-03-13 05:47:42,2023-03-13 18:59:42,0,48,2023-03-13,"(6, 0)"
1513,459,1784,2023-03-13 05:47:42,2023-03-13 18:59:42,0,48,2023-03-13,"(9, 30)"
1518,459,1784,2023-03-13 05:47:42,2023-03-13 18:59:42,0,48,2023-03-13,"(12, 0)"
1525,459,1784,2023-03-13 05:47:42,2023-03-13 18:59:42,0,48,2023-03-13,"(15, 30)"


## Creating fake finished visits - a more complicated example 

In an EHR, information about the patient accumulates as the ED visit progresses. Patients may visit various locations in the ED, such as triage, where their acuity is recorded, and they have various different things done to them, like measurements of vital signs or lab tests. 

The function below returns three fake dataframes, meant to resemble EHR data. 

- hospital visit
- observations - with a single measurement - a triage score - plus a timestamp for when that was recorded
- lab orders - with five types of lab orders plus a timestamp for when that test was requested

The function that creates the fake data returns one triage score for each visit, within 10 minutes of arrival

In [153]:
visits_df, observations_df, lab_orders_df = patient_visits('2023-01-01', '2023-04-01', 25)

print(f'There are {len(observations_df)} triage scores in the observations_df dataframe, for {len(observations_df.visit_number.unique())} visits')
observations_df.head()

There are 2253 triage scores in the observations_df dataframe, for 2253 visits


,visit_number,observation_datetime,triage_score
0,14,2023-01-01 03:35:36.163784,5
1,20,2023-01-01 04:29:37.764451,1
2,1,2023-01-01 05:23:25.286763,2
3,3,2023-01-01 08:10:51.432211,3
4,10,2023-01-01 08:26:20.775693,4


The function that creates the fake data returns a random number of lab tests for each patient, for visits over 2 hours 


In [154]:
print(f'There are {len(lab_orders_df)} lab orders in the dataset, for {len(lab_orders_df.visit_number.unique())} visits')
lab_orders_df.head()

There are 5177 lab orders in the dataset, for 1874 visits


,visit_number,order_datetime,lab_name
0,14,2023-01-01 03:46:03.833071,CBC
1,20,2023-01-01 04:42:28.890734,CBC
2,20,2023-01-01 05:08:21.224579,D-dimer
3,1,2023-01-01 05:23:31.838338,BMP
4,20,2023-01-01 05:45:18.145241,BMP


## Create snapshots from fake data - a more complicated example

A function called `create_snapshots()` will pull information from all three tables. To view the code, click [here](../src/patientflow/generate.py). Note that this function has been designed to work with the fake data generated above. You would need to create your own version of this function, to handle the data you have. 

In [158]:
from datetime import date
start_date = date(2023, 1, 1)
end_date = date(2023, 4, 1)

from patientflow.generate import create_snapshots

# Create snapshots
new_snapshots_df = create_snapshots(visits_df, observations_df, lab_orders_df, prediction_times, start_date, end_date)
new_snapshots_df.head()

,snapshot_date,prediction_time,patient_id,visit_number,is_admitted,age,latest_triage_score,num_cbc_orders,num_d-dimer_orders,num_bmp_orders,num_urinalysis_orders,num_troponin_orders
snapshot_id,,,,,,,,,,,,
0,2023-01-01,"(6, 0)",1658,14,0,30,5.0,1,0,0,0,0
1,2023-01-01,"(6, 0)",238,20,1,61,1.0,1,1,1,0,0
2,2023-01-01,"(6, 0)",354,1,1,86,2.0,0,1,1,0,0
3,2023-01-01,"(9, 30)",114,3,0,33,3.0,0,0,0,0,0
4,2023-01-01,"(9, 30)",497,10,0,59,4.0,0,0,1,0,1


Returning to the example visit above, we can see that at 09:30 on 2023-01-10, the first snapshot for this patient, the triage score had not yet been recorded. This, and the lab orders, were placed between 09:30 and 12:00, so they appear first in the 12:00 snapshot.


In [159]:
new_snapshots_df[new_snapshots_df.visit_number==example_visit_number]

,snapshot_date,prediction_time,patient_id,visit_number,is_admitted,age,latest_triage_score,num_cbc_orders,num_d-dimer_orders,num_bmp_orders,num_urinalysis_orders,num_troponin_orders
snapshot_id,,,,,,,,,,,,
1512,2023-03-13,"(6, 0)",459,1784,0,48,5.0,0,0,0,0,0
1513,2023-03-13,"(9, 30)",459,1784,0,48,5.0,1,0,1,1,0
1518,2023-03-13,"(12, 0)",459,1784,0,48,5.0,1,0,1,1,0
1525,2023-03-13,"(15, 30)",459,1784,0,48,5.0,1,0,1,1,0


## Train a model to predict the outcome of each snapshot

Let's train a model to predict admission for the 9:30 prediction time. We will specify that the triage scores are ordinal, to make use of sklearn's OrdinalEncoder to maintain the natural order of categories. 

We exclude columns that are not relevant to the prediction of probability of admission, including `snapshot_date` and `prediction_time`. Note that here we are trying a different model for each prediction time. That was a design decision, which allows the model to pick up different signals of the outcome at different times of day. You'll see the results of this in later notebooks where I show shap plots for models at different times of day. 

The patient number is important, because if the same patient appears in training and test sets, there is the potential for data leakage. The `create_temporal_splits()` function below will randomly allocate each patient_id to training, validation and test sets, where the probability of being allocated to each is in proportion to the number of visits they made in any of those time periods. 


Note that each record in the snapshots dataframe is indexed by a unique snapshot_id. 

In [ ]:
from datetime import date   
from patientflow.prepare import create_temporal_splits

# set the temporal split
start_training_set = date(2023, 1, 1) 
start_validation_set = date(2023, 2, 15) # 6 week training set 
start_test_set = date(2023, 3, 1) # 2 week validation set 
end_test_set = date(2023, 4, 1) # 1 month test set

# create the temporal splits
train_visits, valid_visits, test_visits = create_temporal_splits(
    snapshots_df,
    start_training_set,
    start_validation_set,
    start_test_set,
    end_test_set,
    col_name="snapshot_date", # states which column contains the date to use when making the splits 
    patient_id="patient_id", # states which column contains the patient id to use when making the splits 
    visit_col="visit_number", # states which column contains the visit number to use when making the splits 

)

# exclude columns that are not needed for training
exclude_from_training_data=['visit_number', 'snapshot_date', 'prediction_time']




Patient Set Overlaps (before random assignment):
Train-Valid: 0 of 1096
Valid-Test: 15 of 784
Train-Test: 62 of 1277
All Sets: 0 of 1540 total patients
Split sizes: [901, 306, 637]


You will need to decide whether you include all snapshots from a single visit into a predictive model. If you do, there will be non-independence in the data. We took the decision to select one visit at random. The function below gives you the option. If you specify `single_snapshot_per_visit` as True, the function will expect a `visit_col` parameter. 


In [178]:
from patientflow.train.classifiers import train_classifier

# train the patient-level model
model = train_classifier(
    train_visits,
    valid_visits,
    test_visits,
    grid={"n_estimators": [30]},
    prediction_time=(9, 30),
    exclude_from_training_data=exclude_from_training_data,
    ordinal_mappings={'latest_triage_score': [1, 2, 3, 4, 5]},
    single_snapshot_per_visit=True,
    visit_col='visit_number', # as we are using a single snapshot per visit, we need to specify which column contains the visit number
    use_balanced_training=True,
    calibrate_probabilities=True
)


In this fake example, the triage score was the most important feature. That is not surprising, as the fake data creation used the triage score to set the values of the other variables 


In [179]:
feature_importances

,Feature,Importance
1,latest_triage_score,0.465822
4,num_urinalysis_orders,0.193781
6,num_troponin_orders,0.083468
5,num_d-dimer_orders,0.082173
0,age,0.073255
2,num_cbc_orders,0.054125
3,num_bmp_orders,0.047376


The output below shows the predicted probability for the first patient we looked at earlier. Note that I'm using the index of `new_snapshots_df` to identify the snapshots I'm interested in. 

In [180]:
snapshot_ids_for_example_patient = new_snapshots_df[new_snapshots_df.visit_number==example_visit_number].index
print(f'Snapshot ids for this patient: {snapshot_ids_for_example_patient.values}')

Snapshot ids for this patient: [1512 1513 1518 1525]


In [181]:
# from the new snapshots_df, find the indices for the visit number we are interested in 
new_snapshots_df.loc[snapshot_ids_for_example_patient]

,snapshot_date,prediction_time,patient_id,visit_number,is_admitted,age,latest_triage_score,num_cbc_orders,num_d-dimer_orders,num_bmp_orders,num_urinalysis_orders,num_troponin_orders
snapshot_id,,,,,,,,,,,,
1512,2023-03-13,"(6, 0)",459,1784,0,48,5.0,0,0,0,0,0
1513,2023-03-13,"(9, 30)",459,1784,0,48,5.0,1,0,1,1,0
1518,2023-03-13,"(12, 0)",459,1784,0,48,5.0,1,0,1,1,0
1525,2023-03-13,"(15, 30)",459,1784,0,48,5.0,1,0,1,1,0


In [182]:
# pass the rows in X (the dataframe prepared for input to the ML alogorithm) to get predictions
preds = model.predict_proba(X.loc[snapshot_ids_for_example_patient])
print(f'Predictions for visit 235 snapshots: {preds[:,1]}')

AttributeError: 'TrainedClassifier' object has no attribute 'predict_proba'

The output above shows that the predicted probability of admission increased for this patient once the triage score was known. 

In [73]:
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': model.feature_importances_}).sort_values('Importance', ascending=False)
feature_importances

,Feature,Importance
1,latest_triage_score,0.465822
4,num_urinalysis_orders,0.193781
6,num_troponin_orders,0.083468
5,num_d-dimer_orders,0.082173
0,age,0.073255
2,num_cbc_orders,0.054125
3,num_bmp_orders,0.047376


## Conclusion

Here I have shown 

* how to create snapshots from finished patient visits
* how to train a very simple model to predict admission at the end of the snapshot. 

A process like this creates a predicted probability of admission for each patient, based on what is known about them at the time of the snapshot. However, bed managers really want predictions for the whole cohort of patients in the ED at a point in time. This is where `patientflow` comes into its own. In the next notebook, I show how to do this. 